In [6]:
import re

# 3rd part imports
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

from dashboard.logic.io import GSHEETS_URL, read_gsheet, comment_button, total_assets, \
    total_value_to_num, getDataFrames

In [23]:
STOCKS_WATCH_URL = "https://docs.google.com/spreadsheets/d/12-GISr1efphjtpuJLCfQzI2akNXxaJ1iabsG24ib71c/edit#gid=845083323"

# Read in summary DF and drop empty rows
df = read_gsheet(
    STOCKS_WATCH_URL, 
    header=None
)

In [69]:
# find df header row index using regex pattern
header_idx = df.apply(lambda x: x.str.contains("Neil's Value", case=False)).any(axis='columns').argmax()

# separate disclaimer and df
df_disclaimer = df.iloc[:header_idx-1, 0]
df_watch = df.iloc[header_idx:,]

# set first row as header & reset row idxs
df_watch.columns = df_watch.iloc[0].values
df_watch = df_watch.iloc[1:].reset_index(drop=True)

# Generate buttons for 'Notes' column
df_watch.Notes[df_watch.Notes.notna()] = df_watch.Notes[df_watch.Notes.notna()].apply(comment_button)
df_watch = df_watch.fillna("")

# Color Ratings based on category
rating_colormap = {'Sig Undervalued':'success', 'Mod Undervalued':'info', 'Fair Value':'light', 'Value Trap?':'danger'}
df_watch["rating_color"] = df_watch.Rating.map(rating_colormap)

# df_watch_cols = df_watch.iloc[0].values
# df_watch = df_watch.iloc[1:]
# df_watch = df_watch.reset_index(drop=True)
# df_watch.columns = df_watch_cols
df_watch

,Company,Ticker,Cur Price,Neil's Value,52W Low,52W High,Dividend,PE,PEG,EPS,Beta,Daily Ch %,Rating,Sector,Notes,rating_color
0,Taiwan Semiconductor Manufacturing,TSM,$94.56,$140.00,$59.43,$128.66,1.87%,14.5,0.68,6.53,1.09,0.07,Sig Undervalued,Technology,,success
1,Facebook,META,$184.23,$385.00,$88.09,$236.86,-,23.0,-,8.03,1.21,-3.86,Sig Undervalued,Communication Services,"<button type=""button"" class=""btn btn-secondary...",success
2,Exelixus,EXEL,$17.74,$29.00,$14.87,$23.40,-,18.7,0.40,0.95,0.72,2.61,Sig Undervalued,Healthcare,,success
3,Adobe,ADBE,$378.33,$650.00,$274.73,$522.48,-,37.5,2.77,10.1,1.23,-1.43,Sig Undervalued,Technology,,success
4,Celanese Corporation,CE,$120.79,$205.00,$86.71,$162.08,2.23%,8.0,7.93,15.09,1.29,-2.36,Sig Undervalued,Chemicals,,success
5,Tesla,TSLA,$198.78,$400.00,$101.81,$384.29,-,54.9,2.70,3.62,2.07,1.00,Sig Undervalued,Consumer Cyclical,"<button type=""button"" class=""btn btn-secondary...",success
6,Paypal,PYPL,$80.04,$230.00,$66.39,$125.28,-,40.7,3.87,1.97,1.30,-3.84,Sig Undervalued,Financial,"<button type=""button"" class=""btn btn-secondary...",success
7,Disney,DIS,$110.34,$160.00,$84.07,$157.50,-,63.1,2.68,1.75,1.28,-1.17,Sig Undervalued,Communication Services,,success
8,Amazon,AMZN,$98.90,$200.00,$81.43,$170.83,-,,3.67,-0.27,1.20,-3.15,Sig Undervalued,Consumer Cyclical,"<button type=""button"" class=""btn btn-secondary...",success
9,Ecolab,ECL,$151.14,$240.00,$131.04,$192.38,1.38%,38.5,4.33,3.92,1.00,-0.70,Sig Undervalued,Basic Materials,,success


In [54]:
df_watch.loc[:"Notes"]

6,Company,Ticker,Cur Price,Neil's Value,52W Low,52W High,Dividend,PE,PEG,EPS,Beta,Daily Ch %,Rating,Sector,Notes,rating_color
0,Taiwan Semiconductor Manufacturing,TSM,$94.56,$140.00,$59.43,$128.66,1.87%,14.5,0.68,6.53,1.09,0.07,Sig Undervalued,Technology,,success
1,Facebook,META,$184.23,$385.00,$88.09,$236.86,-,23.0,-,8.03,1.21,-3.86,Sig Undervalued,Communication Services,"<button type=""button"" class=""btn btn-secondary...",success
2,Exelixus,EXEL,$17.74,$29.00,$14.87,$23.40,-,18.7,0.40,0.95,0.72,2.61,Sig Undervalued,Healthcare,,success
3,Adobe,ADBE,$378.33,$650.00,$274.73,$522.48,-,37.5,2.77,10.1,1.23,-1.43,Sig Undervalued,Technology,,success
4,Celanese Corporation,CE,$120.79,$205.00,$86.71,$162.08,2.23%,8.0,7.93,15.09,1.29,-2.36,Sig Undervalued,Chemicals,,success
5,Tesla,TSLA,$198.78,$400.00,$101.81,$384.29,-,54.9,2.70,3.62,2.07,1.00,Sig Undervalued,Consumer Cyclical,"<button type=""button"" class=""btn btn-secondary...",success
6,Paypal,PYPL,$80.04,$230.00,$66.39,$125.28,-,40.7,3.87,1.97,1.30,-3.84,Sig Undervalued,Financial,"<button type=""button"" class=""btn btn-secondary...",success
7,Disney,DIS,$110.34,$160.00,$84.07,$157.50,-,63.1,2.68,1.75,1.28,-1.17,Sig Undervalued,Communication Services,,success
8,Amazon,AMZN,$98.90,$200.00,$81.43,$170.83,-,,3.67,-0.27,1.20,-3.15,Sig Undervalued,Consumer Cyclical,"<button type=""button"" class=""btn btn-secondary...",success
9,Ecolab,ECL,$151.14,$240.00,$131.04,$192.38,1.38%,38.5,4.33,3.92,1.00,-0.70,Sig Undervalued,Basic Materials,,success
